In [2]:
import praw
from praw.models import MoreComments
import regex
import numpy as np
import pandas as pd
import io
import os

reddit = praw.Reddit(client_id = 'gb2ZbwyyADJBbw', client_secret = 'nGdE_EtfK8pM20dGy369ZmOJxOg', 
                     user_agent = 'SubScraper', username = 'KaranjotSinghV', password = 'jogindersinghvilkhu')

In [ ]:
post_entries = {}
chosen_subject = 'india'

### Here!! Extracting using .search(f"flair:{flair}")

In [ ]:
sub1 = reddit.subreddit(chosen_subject)
postids_sub1 = []

In [ ]:
flairs = ["AskIndia", "Non-Political", "Coronavirus", "Photography", "Science/Technology", "Business/Finance", 
          "Policy/Economy", "Sports", "Food", "Politics", "Scheduled"]

Func for collecting Post Data!

In [ ]:
def collectSubData1(submission, flair):
    
    post = reddit.submission(id = submission) #Accessing subreddit post using id
    
    unique = post.id
    title = post.title
    url = post.url
    body = post.selftext
    score = post.score  
    commentsCount = post.num_comments
    timeStamp = post.created_utc
    
    flairRetrieved = post.link_flair_text
    
    post.comments.replace_more(limit=None)
    
    comments = ''    
    count = 0
    
    for top_level_comment in post.comments:
        
        comments = comments + ' ' + top_level_comment.body
        count += 1     
        
        if(count > 10):
            break
    
    subData = list() #list to store key data of a post
    
    if flairRetrieved == flair: #sanity check
        
        subData.append((unique, title, url, body, score, comments, commentsCount, timeStamp, flairRetrieved))
        post_entries[unique] = subData
        postids_sub1.append(post.id)

In [ ]:
for flair in flairs:
    
    print("Retrieving Posts for " + str(flair))
    get_subreddits = sub1.search(f"flair:{flair}", limit = 300)

    for submission in get_subreddits:

        collectSubData1(submission.id, flair)

In [ ]:
print("Number of posts retrieved - " + str(len(postids_sub1)))

In [ ]:
post_entries[postids_sub1[100]]

### Creating .csv 

In [ ]:
def createcsv():
    upload_count = 0
    
    import csv
    
    location = '/home/vilkhu/MIDAS 2020/Data/'
    filename = input()
    file = location + filename
    
    with open(file, 'w', newline = '') as file:
        
        a = csv.writer(file, delimiter = ',')
        
        headers = ["Post ID", "Title", "URL", "Body", "Score", "Comments", "Comments Count", "Time Stamp", "Flair"]
        
        a.writerow(headers)
        
        for post in post_entries:
            a.writerow(post_entries[post][0])
            upload_count += 1

In [ ]:
createcsv()

In [3]:
data_df = pd.read_csv('Data/PRAW-Data.csv')
data_df.head(5)

,Post ID,Title,URL,Body,Score,Comments,Comments Count,Time Stamp,Flair
0,g014wc,"Lost my Job, Sick Mother and Paralysed Dad, In...",https://www.reddit.com/r/india/comments/g014wc...,Hi....It's really tough time for everyone. I r...,1048,I'm a freelancer. Don't listen to the idiots ...,131,1.586713e+09,AskIndia
1,fxofyu,Why does the government come with a begging bo...,https://www.reddit.com/r/india/comments/fxofyu...,"We have floods, terrorist attacks, famines due...",646,I don't understand why they don't use money f...,204,1.586419e+09,AskIndia
2,g0zlly,Mother's condition is going worse due to hepat...,https://www.reddit.com/r/india/comments/g0zlly...,"Hi folks, I really appreciate the warm respons...",763,If anyone knows who is influential on Twitter...,94,1.586842e+09,AskIndia
3,fvy95j,Men who are 30+ and have decided not to get ma...,https://www.reddit.com/r/india/comments/fvy95j...,The corona virus has given me some time to thi...,266,Plan your finances. Work and enjoy in your ow...,206,1.586178e+09,AskIndia
4,g1lmhg,[Please Advice] Reality punched me in the face...,https://www.reddit.com/r/india/comments/g1lmhg...,"Sorry Reddit, this post is going to be long. P...",430,One thing I will say is don't try to clear of...,83,1.586928e+09,AskIndia


In [4]:
print("Total number of posts retrieved - " + str(data_df.shape[0]))

Total number of posts retrieved - 2656


In [5]:
data_df["Flair"].value_counts()

Politics              249
Coronavirus           249
Scheduled             246
Science/Technology    245
Policy/Economy        245
Non-Political         244
Business/Finance      243
Food                  242
Photography           241
AskIndia              235
Sports                217
Name: Flair, dtype: int64

### Here!! Extracting hot, new, rising, top posts

This is another method of extracting posts, the posts are categorized as hot, new, rising and top. This method was not opted because the number of posts retrieved for each Flair is not balanced.

In [ ]:
postids_sub2 = []

In [ ]:
sub2 = reddit.subreddit(chosen_subject).hot(limit = None)

In [ ]:
for submission in sub2:
    
    postids_sub2.append(submission.id)
    
postids_sub2 = list(np.unique(postids_sub2))
postids_sub2 = [elem for elem in postids_sub2 if elem not in postids_sub1]
print("Number of IDs retrieved - " + str(len(postids_sub2)))

Func for collecting Post Data!

In [ ]:
def collectSubData2(submission):
    
    post = reddit.submission(id = submission) #Accessing subreddit post using id
    
    post.comments.replace_more(limit=None)
    
    comments = ''    
    count = 0
    
    for top_level_comment in post.comments:
        
        comments = comments + ' ' + top_level_comment.body
        count += 1     
        
        if(count > 10):
            break
    
    unique = post.id
    
    title = post.title
    
    url = post.url
    
    body = post.selftext
    
    timeStamp = post.created_utc
    
    commentsCount = post.num_comments
    
    score = post.score
         
    flair = post.link_flair_text
    
    subData = list() #list to store key data of a post
    
    if flair in flairs:
    
        subData.append((unique, title, url, body, score, comments, commentsCount, timeStamp, flair))
        post_entries[unique] = subData

In [ ]:
for count, submission in enumerate(postids_sub2):
    
    print('Collecting data for post - ' + str(count))
    
    collectSubData2(submission)